In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP 上的端到端机器学习：MLOps 阶段 2：实验：使用 Vertex AI 训练开始

您可以点击以下链接查看更多信息：
- [GitHub链接](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training.ipynb)
- [Colab链接](https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training.ipynb)
- [Vertex AI Workbench链接](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training.ipynb)

## 概述

本教程演示了如何在谷歌云上使用Vertex AI进行端到端的生产环境MLOps。本教程涵盖了第二阶段：实验：开始使用Vertex AI训练。

### 目标

在本教程中，您将学习如何在使用 `Vertex AI` 进行训练时使用 `Vertex AI 训练` 来训练自定义模型。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI 训练`

执行的步骤包括：

- 使用单个 Python 脚本进行训练。
- 使用 Python 包进行训练。
- 使用自定义训练图像进行训练。
- 设计一个训练包。

### 推荐

在 Google Cloud 上进行端到端的 MLOps 时，以下是为选择自定义模型训练的 SDK 方法的最佳实践：

**您有一个 Python 训练包并且正在使用预构建的 Google 容器**

CustomPythonPackageTrainingJob

**您有一个包含 Python 训练包的 Docker 镜像**

CustomContainerTrainingJob

**您有一个单独的训练脚本（不是包）并且正在使用预构建的 Google 容器**

CustomTrainingJob

**其他**

CustomJob

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。 这个教程中使用的数据集是内置于TensorFlow中的版本。 训练好的模型可以预测房屋的中位价格，单位为1千美元。

### 成本

这个教程使用了Google Cloud的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 来根据您的预期使用量生成成本估算。

安装所需的软件包以执行此笔记本

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q

### 重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*确保如果你有这个选项的话，你在一个 GPU 运行时中运行这个笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置你的 Google Cloud 项目

**无论你的笔记本环境如何，下面的步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建帐户时，你会获得$300 的免费信用用于计算/存储成本。

2. [确保你的项目已启用结算功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI APIs，Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果你在本地运行这个笔记本，你需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入你的项目ID。然后运行该单元格，确保 Cloud SDK 对这个笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并且通过 `$` 为前缀的 Python 变量进行插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于整个笔记本中的操作。以下是Vertex AI支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会在使用Vertex AI进行训练时使用多区域存储桶。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加现场教程会话，您可能会使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证，通过oAuth授权您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中输入您服务帐号密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，都需要执行以下步骤。**

当您初始化 Python 版本的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在会话之间保留的地方。

请在下面设置您的云存储桶的名称。存储桶名称在所有谷歌云项目中必须是全局唯一的，包括您所在组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶中的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 用于 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以便使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU的GPU容器映像分配给每个VM，可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个运行在CPU上的容器映像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3之前的GPU支持版本将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由生成在serving函数中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建容器

设置用于训练和预测的预构建 Docker 容器镜像。


最新列表请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


最新列表请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9 GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]个

*注意：以下不支持用于训练:*

 - `standard`：2个vCPUs
 - `highcpu`：2、4、8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

自定义训练作业

当您有一个单独的Python训练脚本并且使用Google预构建容器时，可以使用`CustomTrainingJob`。通常情况下，当开始一个简单实验时，会使用这种方法。

### 创建和运行自定义训练任务

要训练一个自定义模型，需要执行两个步骤：1) 创建一个自定义训练任务，和 2) 运行任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要以下参数：

- `display_name`：用于自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP
REQUIREMENTS = ["tensorflow==2.3"]

job = aiplatform.CustomTrainingJob(
    display_name=DISPLAY_NAME,
    script_path="task.py",
    requirements=REQUIREMENTS,
    container_uri=TRAIN_IMAGE,
)

#### Task.py内容

在下一个单元格中，您可以编写训练脚本task.py的内容。您不会训练模型，而是为了演示目的而保持简单。

总结如下：

- 从命令行获取保存模型产物的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 在保存模型产物的目录中打开一个名为"test.txt"的文件。
- 在文件中写入"hello world"。

In [ ]:
%%writefile task.py
import argparse
import os
import tensorflow as tf

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

with tf.io.gfile.GFile(args.model_dir + '/test.txt', 'w') as f:
    f.write('hello world\n')

运行自定义训练作业

接下来，您可以通过调用`run`方法运行自定义作业来启动训练作业，参数如下：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `sync`：是否阻塞直到作业完成。

默认情况下，训练实例将使用 CPU。如果要使用 GPU 进行训练，则添加以下参数：

- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：附加到工作人员副本的加速器数量。

如果要覆盖命令行参数 `--model_dir`，请使用以下参数指定写入模型工件的位置。请注意，Vertex AI 训练服务将通过环境变量 `AIP_MODEL_DIR` 将此位置传递给训练脚本：

- `base_output_dir`：用于写入模型工件的 Cloud Storage 位置。

如果您同时使用 Vertex AI TensorBoard 进行训练，请添加以下参数。请注意，Vertex AI 训练服务将通过环境变量 `AIP_TENSORBOARD_LOG_DIR` 将 TensorBoard 日志文件的位置传递给训练脚本：

- `tensorboard`：Vertex AI TensorBoard 实例的资源名称。
- `service_account`：具有访问 Cloud Storage 和 Vertex AI TensorBoard 权限的服务帐号。

笔记本后面还涵盖了更多与以下相关的其他参数：

- 使用 Vertex Dataset
- 将模型上传到 Vertex 模型资源

In [ ]:
CMDARGS = [
    "--model-dir=" + BUCKET_URI,
]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

! gsutil cat {BUCKET_URI}/test.txt

### 删除自定义训练任务

训练任务完成后，您可以使用`delete()`方法来删除训练任务。在完成之前，可以使用`cancel()`方法取消训练任务。

In [ ]:
job.delete()

## 自定义Python包训练任务

当您拥有一个包含脚本的Python训练包并使用Google预构建的容器时，可以使用`CustomPythonPackageTrainingJob`。通常情况下，人们会在实验阶段和后续的正式训练中使用该方法。

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要以下参数：

- `display_name`：自定义训练任务的易读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball形式存储。
- `python_module_name`：Python包中训练脚本的相对路径。

*注意*：没有`requirements`参数。您可以在Python包的`setup.py`脚本中指定所有要求。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_boston.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
)

### 检查培训包

#### 包布局

在开始培训之前，您将查看Python包是如何为自定义培训作业组装的。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境的说明书。

文件`trainer/task.py`是执行自定义培训作业的Python脚本。*注意*，当我们在工作池规范中引用它时，我们用点(`trainer.task`)代替目录斜杠，并且删除文件后缀(`.py`)。

#### 包组装

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py的内容

在下一个单元格中，你将编写训练脚本task.py的内容。你不会训练一个模型，而是为演示目的而保持简单。

总结一下：

- 从命令行（`--model_dir`）获取要保存模型工件的目录，如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从命令行获取要运行的epoch数量（`--model_dir`）。
- 在要保存模型工件的目录中打开一个名为"test.txt"的文件。
- 每个epoch都重复将"hello world"追加到文件中。

In [ ]:
%%writefile custom/trainer/task.py
import argparse
import os
import tensorflow as tf

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
args = parser.parse_args()

with tf.io.gfile.GFile(args.model_dir + '/test.txt', 'w') as f:
    for epoch in range(args.epochs):
        f.write('hello world\n')

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

运行定制的Python软件包训练作业

接下来，通过调用方法`run()`来运行定制的作业以启动训练作业。参数与运行CustomTrainingJob时相同。

In [ ]:
CMDARGS = ["--model-dir=" + BUCKET_URI, "--epochs=5"]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

! gsutil cat {BUCKET_URI}/test.txt

### 删除自定义训练作业

在训练作业完成后，您可以使用"delete()"方法来删除训练作业。在完成之前，您可以使用"cancel()"方法来取消训练作业。

In [ ]:
job.delete()

自定义容器训练任务

当您有一个包含训练程序的Docker镜像时，可以使用`CustomContainerTrainingJob`。通常情况下，人们会同时用于实验和后续正式训练。

创建一个Docker文件

为了使用自己的定制训练容器，您需要构建一个Docker文件，并将您的训练脚本嵌入到容器中。

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装Python包。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境的说明。

文件`trainer/task.py`是执行自定义培训任务的Python脚本。*注意*，当在工作池规范中引用它时，我们将目录斜杠替换为点（`trainer.task`）并删除文件后缀（`.py`）。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您将编写训练脚本task.py的内容。您不会训练一个模型，而是为演示目的保持简单。

总结一下：

- 从命令行（`--model_dir`）获取保存模型工件的目录，如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从命令行获取要运行的轮数 (`--model_dir`)。
- 在保存模型工件的目录中打开一个名为"test.txt"的文件。
- 每个轮数重复向文件中添加"hello world"。

In [ ]:
%%writefile custom/trainer/task.py
import argparse
import os
import tensorflow as tf

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
args = parser.parse_args()

with tf.io.gfile.GFile(args.model_dir + '/test.txt', 'w') as f:
    for epoch in range(args.epochs):
        f.write('hello world\n')

#### 编写 Docker 文件内容

将您的代码容器化的第一步是创建一个 Docker 文件。在您的 Docker 文件中，您将包含运行容器映像所需的所有命令。它将安装您正在使用的所有库，并设置您的训练代码的入口点。

1. 从 TensorFlow 仓库安装预定义的深度学习映像容器。
2. 复制 Python 训练代码，稍后会显示。
3. 将入口点设置为 Python 训练脚本 `trainer/task.py`。请注意，ENTRYPOINT 命令中省略了 `.py`，因为它是隐含的。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建容器本地

接下来，您将为您的客户容器提供一个名称，当您将其提交到谷歌容器库时将使用该名称。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/boston:v1"

接下来，建立容器。

In [ ]:
if not IS_COLAB:
    ! docker build custom -t $TRAIN_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

#### 在本地测试容器

在您的笔记本实例中运行容器，以确保它能正常工作。您将运行5个时代。

In [ ]:
if not IS_COLAB:
    ! docker run $TRAIN_IMAGE --epochs=5 --model-dir=./

#### 注册自定义容器

当您在本地运行容器完成后，将其推送到 Google 容器注册表。

In [ ]:
if not IS_COLAB:
    ! docker push $TRAIN_IMAGE

在Colab上执行

In [ ]:
%%bash -s $IS_COLAB $TRAIN_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build custom -t $2
docker run $2 --epochs=5 --model-dir=./
docker push $2
kill $(jobs -p)

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练作业，2）运行该作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，其中包括以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。

- `command`：容器内要调用的命令（例如，解释器）和脚本。

*注意：* 在容器内可以重写要调用的解释器和脚本（即 ENTRYPOINT）。

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="boston_" + TIMESTAMP,
    container_uri=TRAIN_IMAGE,
    command=["python3", "trainer/task.py"],
)

print(job)

#### 运行自定义容器训练任务

接下来，您可以通过调用方法 `run()` 来运行自定义任务以开始训练任务。参数与运行 CustomTrainingJob 时相同。

In [ ]:
CMDARGS = ["--model-dir=" + BUCKET_URI, "--epochs=5"]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

! gsutil cat {BUCKET_URI}/test.txt

删除自定义训练作业

在训练作业完成后，您可以使用`delete()`方法删除训练作业。在完成之前，可以使用`cancel()`方法取消训练作业。

In [ ]:
job.delete()

## 模型保存和转换为模型资源。

Vertex AI训练将自动将您训练的模型转换为 Vertex AI 模型资源，通过添加以下附加步骤：

- 在创建自定义训练作业（例如 CustomTrainingJob）时指定以下附加参数：

    - `base_output_dir`：用于存储训练模型工件的 Cloud Storage 位置。
    - `model_serving_container_image_uri`：模型的部署容器。

- 在运行作业时指定以下附加参数：

    - `model_display_name`：模型资源的显示名称。

- 在训练脚本中，将模型保存到由环境变量`AIP_MODEL_DIR`指定的位置。

服务将根据`base_output_dir`的 Cloud Storage 位置来设置环境变量`AIP_MODEL_DIR`的值。在训练脚本成功完成后，服务将在此位置查看模型工件，并将模型工件上传到 Vertex AI 模型资源中。

使用`run()`方法的响应将在同步运行时是模型资源。在异步运行时，响应是一个 LRO，可以在其上执行`wait()`操作。完成后，响应对象将重定向到模型资源。

#### Task.py 文件内容

在下一个单元格中，您可以查看训练脚本 task.py 的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 训练模型（`fit()`）并指定训练周期`args.epochs`。
- 将训练好的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值写入到指定的参数文件中（`f.write(str(params))`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储到您的云存储桶

接下来，您将培训文件夹打包成压缩的tar包，然后存储到您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

设置预构建的容器

为训练设置预构建的Docker容器映像。

- 将变量 `TF` 设置为容器映像的TensorFlow版本。例如，`2-1` 表示版本2.1，`1-15` 表示版本1.15。以下列表显示了一些可用的预构建映像：


查看最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练作业，2) 运行这个作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类来创建一个自定义训练作业，具有以下参数：

- `display_name`：自定义训练作业的人类可读名称。
- `container_uri`：训练容器镜像。
- `python_package_gcs_uri`：Python训练包的位置，以tarball格式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意：* 没有requirements参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "boston_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_boston.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 准备您的命令行参数

现在定义您自定义训练容器的命令行参数：

- `args`：传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：在我们的演示中，我们使用这个命令行参数来指定存储模型构件的位置。
      - 直接：您将云存储位置作为命令行参数传递给您的训练脚本（设置变量 `DIRECT = True`），或者
      - 间接：服务将云存储位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）. 在这种情况下，您告诉该服务作业说明书中的模型构件位置。
  - `"--epochs=" + EPOCHS`：训练的时期数。
  - `"--steps=" + STEPS`：每个时期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练作业

接下来，通过调用 `run` 方法并传入以下参数来运行自定义作业以开始训练作业：

- `model_display_name`：`Model` 资源的人类可读名称。
- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 为单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `base_output_dir`：写入模型工件的 Cloud Storage 位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="boston_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="boston_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

### 列出一个自定义训练工作

In [ ]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者可以在`run()`方法中将参数`sync`设置为`True`以阻塞，直到自定义训练作业完成。

In [ ]:
model.wait()

## 自定义训练脚本建议

从实验阶段向正式阶段转变时，通常会以一种便于转换为自动化流水线的方式构建训练脚本。典型做法是将自定义训练脚本中的不同任务分开放在单独的Python模块中，例如：

- `data.py`: 数据流水线。
- `model.py`: 构建或检索模型架构。
- `train.py': 训练流水线。
- `eval.py`: 评估训练模型。
- `task.py`: 入口点和脚本驱动程序。

In [ ]:
%%writefile custom/trainer/task.py
from . import data
from . import model
from . import train
from . import eval

parser = argparse.ArgumentParser()
# ...
args = parser.parse_args()

train, val, test = data.get(args.batch_size, ...)

model = model.get(args.lr, ... )

history = train.train(model, train, val, args.epochs, ...)

metrics = eval.evaluate(model, eval)

model.save(args.model_dir)

清理

要清理本项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_job = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_job:
    try:
        job.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}